In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm.notebook import tqdm
import time
import urllib.request
import requests
import urllib
import re

In [4]:
station_list = ['홍대입구역', '강남역', '명동역', '신촌역', '서울대입구역', '사당역', '혜화역', '가산디지털단지역', '서울역', '청담역', '종로3가역', '을지로입구역', '동대문역사문화공원역',   '성신여대입구역', '이태원역', '성수역',  '영등포역', '여의도역', '잠실역', '압구정역', '신사역', '판교역']
def input_keys():
    global stop_check
    key = input("s를 누르면 STOP : ")
    if key == ('s' or 'S'):
        stop_check = 1

# 검색 키워드를 받는 함수
def initkeyword(driver):
    print("더 보기를 누르고 '로그인이 필요합니다' 영역을 눌러 로그인 진행해주세요.")
    driver.get("https://map.naver.com/v5/search/")
    while True:
        key = input("로그인이 완료됐으면 'Y'를 입력해주세요. :")
        if key == 'Y' or key == 'y':
            #print('검색 키워드 입력')
            #encText = urllib.parse.quote(f"{input('검색 키워드 : ')}")
            encText = urllib.parse.quote(station_list[0] + ' 음식점')
            driver.get("https://map.naver.com/v5/search/" + encText)
            break
        else:
            print('잘못된 입력. 다시 실행해주세요.')
            break
    return

# 맨 처음 스크롤을 끝까지 내려서 한 페이지 내 모든 음식점 HTML 태그 불러옴.
def init_scrollOnce(driver):
    driver.switch_to.default_content()
    next_iframe = driver.find_element(By.ID,"searchIframe")
    driver.switch_to.frame(next_iframe)

    for idx in tqdm(range(1,55)):
        try:
            ele = driver.find_element(By.XPATH,f'''//*[@id="_pcmap_list_scroll_container"]/ul/li[{idx}]/div[1]/a[1]/div/div/span[1]''');
            driver.execute_script("arguments[0].scrollIntoView(true);",ele);
            driver.implicitly_wait(1)
        except:
            continue
    return 


# 가져올 장소
홍대입구역, 강남역, 명동역, 신촌역, 서울대입구역, 사당역, 혜화역, 가산디지털단지역, 서울역, 청담역, 종로3가역, 을지로입구역, 동대문역사문화공원역,   성신여대입구역, 이태원역, 성수역,  영등포역, 여의도역, 잠실역, 압구정역, 신사역
# 가져올 항목
이름, 역, 구, 카테고리, 방문자리뷰갯수, 블로그리뷰갯수, 평점, 주소, 영업시간, 부가서비스, 방문자리뷰글

In [5]:
#station_list = ['홍대입구역', '강남역', '명동역', '신촌역', '서울대입구역', '사당역', '혜화역', '가산디지털단지역', '서울역', '청담역', '종로3가역', '을지로입구역', '동대문역사문화공원역',   '성신여대입구역', '이태원역', '성수역',  '영등포역', '여의도역', '잠실역', '압구정역', '신사역', '판교역']
station_list = ['여의도역']

## 최종 코드

In [7]:
global stop_check
from threading import Thread
from selenium.common.exceptions import NoSuchElementException

shop_name_list = []
close_station_list = []
gu_list = []
category_list = []
visit_review_list = []
blog_review_list = []
rating_list = []
address_list = []
open_time_list = []
service_list = []
review_contents_list = []
detailed_review = []
latitude = []
longitude = []

driver = webdriver.Chrome(service=Service("../../EDA/driver/chromedriver"))

def main_action():
    
    
    global stop_check
    stop_check = 0
    
    for i in range(len(station_list)):
        # 다음 역 근처 음식점 검색
        driver.switch_to.default_content()
        input_tab = driver.find_element(By.CLASS_NAME, "input_search")
        input_clear  = driver.find_element(By.CLASS_NAME, 'btn_clear')
        input_clear.click()
        time.sleep(0.5)
        input_tab.send_keys(station_list[i] + " 음식점")
        input_tab.send_keys(Keys.RETURN)
        time.sleep(0.5)
        
        while stop_check != 1:    
            for idx in tqdm(range(1,55)):
                # 음식점 선택 프레임으로 iframe 이동
                driver.switch_to.default_content()
                next_iframe = driver.find_element(By.ID,"searchIframe")
                driver.switch_to.frame(next_iframe)
                
                # 음식점 링크 클릭 후 상세창 열기(li[idx]에서 idx 변경(1~53))
                time.sleep(1)
                try:
                    ele = driver.find_element(By.XPATH,f'''//*[@id="_pcmap_list_scroll_container"]/ul/li[{idx}]/div[1]/a[1]/div/div/span[1]''');
                    driver.execute_script("arguments[0].scrollIntoView(true);",ele);
                except:
                    continue
                #ele.click()
                WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH,f'''//*[@id="_pcmap_list_scroll_container"]/ul/li[{idx}]/div[1]/a[1]/div/div/span[1]''')))
                time.sleep(0.5)
                try:
                    ele.click()
                except:
                    continue
                
                # 상세창으로 이동
                time.sleep(1)
                driver.implicitly_wait(3)
                driver.switch_to.default_content()
                time.sleep(1)
                next_iframe = driver.find_element(By.ID,"entryIframe")
                driver.switch_to.frame(next_iframe)
                driver.execute_script("window.scrollTo(0, 0)")
                driver.implicitly_wait(3)
                time.sleep(1)
                
                # 이름, 카테고리
                driver.execute_script("window.scrollTo(0, 0)")
                try:
                    shop_box = driver.find_element(By.CLASS_NAME, 'YouOG')
                    shop_name =  shop_box.find_element(By.CLASS_NAME, "Fc1rA").text
                    category = shop_box.find_element(By.CLASS_NAME, 'DJJvD').text
                except:
                    continue 
                #print(shop_name)
                #print(category)

                # 평점, 방문자리뷰갯수, 블로그리뷰갯수
                review_box = driver.find_elements(By.CLASS_NAME, 'PXMot')
                if len(review_box) > 2:
                    try:
                        rating = review_box[0].find_element(By.TAG_NAME, 'em').text
                        visit_review = review_box[1].find_element(By.TAG_NAME, 'em').text
                        blog_review = review_box[2].find_element(By.TAG_NAME, 'em').text
                    except:
                        continue
                else:
                    rating = ''
                    try:
                        visit_review = review_box[0].find_element(By.TAG_NAME, 'em').text
                        blog_review = review_box[1].find_element(By.TAG_NAME, 'em').text
                    except:
                        continue
                #print(rating)
                #print(visit_review)
                #print(blog_review)

                # 주소, 구, 역, 영업시간, 부가서비스
                try:
                    address = driver.find_element(By.CLASS_NAME, 'LDgIH').text
                    #print(address)
                    gu = address.split()[1]
                    #print(gu)
                except:
                    continue
                
                try:
                    station_box = driver.find_element(By.CLASS_NAME, 'nZapA')
                    close_station = station_box.find_element(By.TAG_NAME, 'strong').text
                except:
                    continue
                
                #print(close_station)
                try:
                    time_btn_box = driver.find_element(By.CLASS_NAME, 'y6tNq')
                    time_btn = time_btn_box.find_element(By.CLASS_NAME, '_UCia')
                    time_btn.click()
                except:
                    continue

                open_time = []
                time_box = driver.find_elements(By.CLASS_NAME, 'A_cdD')
                for i in range(1, len(time_box)):
                    if '-' in time_box[i].text or ':' in time_box[i].text:
                        open_time.append(time_box[i].text)
                open_time = str(open_time).replace('\\n', ' ').replace('[', '').replace(']', '').replace("'", '')
                #print(open_time)

                services = driver.find_elements(By.CLASS_NAME, 'vV_z_')
                service_words = []
                for service in services:
                    service = service.text 
                    if '단체석' in service:
                        service_words.append('단체석')
                    if '포장' in service:
                        service_words.append('포장')
                    if '예약' in service:
                        service_words.append('예약')
                    if '배달' in service:
                        service_words.append('포장')
                    if '무선 인터넷' in service:
                        service_words.append('무선 인터넷')
                    if '남/녀 화장실 구분' in service:
                        service_words.append('남/녀 화장실 구분')
                service_words = str(service_words).replace('[', '').replace(']', '').replace("'", "")
                #print(service_words)     
                
                # 리뷰 클릭
                review = driver.find_element(By.CLASS_NAME, 'place_fixed_maintab')
                i = 1
                while True:
                    review1 = review.find_element(By.XPATH, f'''//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[{i}]''')
                    if review1.text == "리뷰":
                        break
                    else: i+=1
                review1.click()
                #펼치기
                time.sleep(1)
                try:
                    spread = driver.find_element(By.CLASS_NAME, 'Tvx37')
                    while True:
                        try:
                            spread.click()
                        except:
                            break
                except:
                    continue
                time.sleep(0.3)
                review2 = driver.find_element(By.CLASS_NAME, 'uNsI9')
                # 자료 전처리
                reviews = review2.text
                reviews = reviews.split('\n')
                to_remove = ["이 키워드를 선택한 인원"]
                review_list = list(filter(lambda item: item not in to_remove, reviews))
                dic1 = review_list[::2]
                dic2 = review_list[1::2]
                review_dic = dict(zip(dic1, dic2))
                review = str(review_dic).replace("{", '').replace('}', '').replace('"', '').replace("'", '')
                #print(review)   
                
                # 개인 리뷰
                try:
                    detail_re1 = driver.find_element(By.CLASS_NAME, 'eCPGL') # 리뷰 상위 
                    detail_re3 = driver.find_element(By.CLASS_NAME, 'fvwqf') # 전체리뷰 더보기
                except:
                    continue
                
                #전체 리뷰 펼치기
                while True:
                    detail_re2 = detail_re1.find_elements(By.CLASS_NAME, 'YeINN') # 전체 리뷰
                    try:
                        detail_re3.click()
                        if len(detail_re2) >=100:
                            break
                    except:
                        break
                #단일 리뷰 펼치기
                try:
                    driver.execute_script("window.scrollTo(0, 0);")
                    detail_re4 = detail_re1.find_elements(By.CLASS_NAME, 'rvCSr')
                    a = 1
                except:
                    continue
                
                while True:
                    try:
                        detail_re4[a].click()
                    except:
                        break
                    driver.implicitly_wait(3)
                    a+=1
                    
                    if a>=len(detail_re4):
                        break
                    
                # 리뷰 가져오기(빈 글은 무시하고)
                all_detail = []
                b = 0
                while b < len(detail_re2):
                    try:
                        # Check if b is a valid index
                        if b >= 0 and b < len(detail_re2):
                            detail_re5 = detail_re2[b].find_element(By.CLASS_NAME, 'xHaT3')
                            all_detail.append(detail_re5.text)
                    except NoSuchElementException:
                        # Handle the case when the element is not found (optional)
                        pass
                    
                    b += 1
                all_detail = str(all_detail).replace('[', '').replace(']', '').replace('\\n', ' ').replace("'", "")
                
                shop_name_list.append(shop_name)
                close_station_list.append(close_station)
                gu_list.append(gu)
                category_list.append(category)
                visit_review_list.append(visit_review)
                blog_review_list.append(blog_review)
                rating_list.append(rating)
                address_list.append(address)
                open_time_list.append(open_time)
                service_list.append(service_words)
                review_contents_list.append(review)
                detailed_review.append(all_detail)
                
                print(shop_name, close_station, gu, category, visit_review, blog_review, rating, address, open_time, service_words, review, all_detail)
            
            
            # 음식점 선택 프레임으로 iframe 이동
            driver.switch_to.default_content()
            next_iframe = driver.find_element(By.ID,"searchIframe")
            driver.switch_to.frame(next_iframe)
                
            next_page = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#app-root > div > div.XUrfU > div.zRM9F > a:nth-child(7)')))
            #next_page = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH,'''//*[@id="app-root"]/div/div[4]/div[2]/a[7]''')))
            usable = next_page.get_attribute('aria-disabled')
            # usable = 'true'                                                                                              
            if usable == 'false':
                ActionChains(driver).move_to_element(next_page).perform()
                next_page.send_keys(Keys.ENTER)
                # next_page.click()
                driver.implicitly_wait(2)
            else:
                stop_check = 1
                print('Sucessfully Complete !!')
                break
    return 

from pynput import keyboard
def on_press(key):
    print('Key %s pressed' % key)

def on_release(key):
    global stop_check
    # print('Key %s released' %key)
    if key == keyboard.Key.esc: #esc 키가 입력되면 종료
        stop_check = 1
        return False

# def stop_func():
#     with keyboard.Listener(
#         on_press=on_press,
#         on_release=on_release) as listener:
#         listener.join()        
        
import sys
if __name__ == '__main__':
    initkeyword(driver)
    driver.implicitly_wait(2)
    init_scrollOnce(driver)
    print('종료를 원하면 esc를 누르세요.')
    #t2 = Thread(target=main_action())
    main_action()
    # t1 = Thread(target=stop_func())
    # with keyboard.Listener(
    #     on_press=on_press,
    #     on_release=on_release) as listener:
    #     listener.join()    
    # t1.start()
    # t1.join()
    #t2.start()


더 보기를 누르고 '로그인이 필요합니다' 영역을 눌러 로그인 진행해주세요.


  0%|          | 0/54 [00:00<?, ?it/s]

종료를 원하면 esc를 누르세요.


  0%|          | 0/54 [00:00<?, ?it/s]

민소한우 금융로점 여의도역 영등포구 소고기구이 459 77 4.45 서울 영등포구 국제금융로6길 26 노총회관 지하 1층 민소 금융로점 월 11:00 - 22:00 14:30 - 17:00 브레이크타임 14:00, 21:30 라스트오더, 화 11:00 - 22:00 14:30 - 17:00 브레이크타임 14:00, 21:30 라스트오더, 수 11:00 - 22:00 14:30 - 17:00 브레이크타임 14:00, 21:30 라스트오더, 목 11:00 - 22:00 14:30 - 17:00 브레이크타임 14:00, 21:30 라스트오더, 금 11:00 - 22:00 14:30 - 17:00 브레이크타임 14:00, 21:30 라스트오더 단체석, 포장, 예약, 포장, 무선 인터넷, 남/녀 화장실 구분 음식이 맛있어요: 147, 친절해요: 65, 매장이 넓어요: 50, 단체모임 하기 좋아요: 50, 매장이 청결해요: 39, 재료가 신선해요: 30, 특별한 메뉴가 있어요: 28, 특별한 날 가기 좋아요: 24, 양이 많아요: 17, 고기 질이 좋아요: 14, 가성비가 좋아요: 14, 인테리어가 멋져요: 7, 주차하기 편해요: 6, 직접 잘 구워줘요: 5, 혼밥하기 좋아요: 3, 화장실이 깨끗해요: 2, 뷰가 좋아요: 2, 좌석이 편해요: 1, 음식이 빨리 나와요: 1, 룸이 잘 되어있어요: 1 제가 좋아하던 한우등심불고기전골이 업데이트되었네요. 고기 땟깔이 넘좋아졌어요. 야채랑 고기랑 부드럽고 맛있어요~~, 한우국수전골 추천합니다. 고기 땟깔좋고. 국물 맛있고  마지막에는 죽도해주고 든든하게 먹었어요 점심시간 바쁜데도 신경써서 해주셔서 좋았어요, 저는 여의도 25년차 직장인이구요 민소중에 여기가 젤 맛없는데 후기 올라온거 보니 안좋은거 올라오면 계속 덮 으시나 봅니다 열심히 하시는데 이런거 열심히 하... 내용 더보기, 직접 다 구워주시고, 고기가 넘 맛있네요!, 점심에 고기 먹기엔 별로 좋지 않네요. 고기는 맛있었어요., 런치정식 맛있게 잘먹었어요, 매운 갈비찜 

In [24]:
print(len(shop_name_list), shop_name_list)
print(len(close_station_list), close_station_list)
print(len(gu_list), gu_list)
print(len(category_list), category_list)
print(len(visit_review_list), visit_review_list)
print(len(blog_review_list), blog_review_list)
print(len(rating_list), rating_list)
print(len(address_list), address_list)
print(len(open_time_list), open_time_list)
print(len(service_list), service_list)
print(len(review_contents_list), review_contents_list)
print(len(detailed_review), detailed_review)

294 ['마르셀', '오복수산대구', '진주집', '옥된장 여의도점', '미도인 파이낸스여의도', '세상의모든아침', '을지다락 여의도', '후라토식당 여의도직영점', '오복수산 여의도점', '방이편백 육분삼십 본점', '이타마에', '테이스팅룸 IFC몰', '사위식당 여의도점', '콘타이 IFC몰점', '콘래드서울 제스트', '사대부집 곳간', '브루클린더버거조인트 여의도점', '사카바 단단', '콘래드서울 37그릴앤바', '별미볶음점', '진진만두 여의도점', '가양칼국수버섯매운탕', '이타마에', '퀴진 알', '도쿄등심 여의도점', '고우가 여의도점', '야마야 여의도점', '바스버거 여의도점', '탭퍼블릭 여의도점', '모도우 여의도점', '여의도 스마일삼겹', '콘래드서울 버티고', '고집132 여의도점', '순대실록 IFC몰점', '아루히', '훠궈야 여의도IFC몰점', '서울로인 여의도점', '아웃백스테이크하우스 여의도점', '차알 여의도 IFC몰점', '한암동 여의도점', '온더보더 여의도IFC점', '콘래드서울 아트리오', '한미옥 여의도직영점', '송림', '여의도 셋째집', '청수우동메밀냉면', '노티드 여의도 IFC', '본보야지', '히노노리 오투타워점', '오복수산참치 여의도점', '선주후면 어만두', '더플레이스 여의도IFC점', '와인주막차차 여의도점', '에덴식당', '창고43 본점', '예성가든', '카페꼼마&얀 쿠브레 여의도점', '아그라 여의도IFC몰점', '판다익스프레스 IFC몰점', '긴자', '마마된장 여의도 IFC몰점', '진가와 여의도점', '제일제면소 여의도IFC몰점', '오늘은즉떡 본점', '함루 여의도점', '알로하포케 IFC몰점', '닥터로빈 파크센터점', '일호면옥', '쓰리버즈 IFC점', '스시젠', '레어커피', '점보씨푸드 IFC몰점', '카츠8 IFC몰점', 'TENUTO', '차이나플레인 전경련점', '오토김밥 여의도점', '폴앤폴리나 여의도점', '피그인더가든 여의도점', '단디만', '경복

In [25]:
import pandas as pd
df = pd.DataFrame({'이름': shop_name_list,
                   '역': close_station_list,
                   '구': gu_list,
                   '카테고리': category_list,
                   '방문자_리뷰_개수': visit_review_list,
                   '블로그_리뷰_개수': blog_review_list,
                   '평점': rating_list,
                   '주소': address_list,
                   '영업시간': open_time_list, 
                   '부가서비스': service_list,
                   '방문자_리뷰_글': review_contents_list,
                   '개인리뷰' : detailed_review
                   })
df

,이름,역,구,카테고리,방문자_리뷰_개수,블로그_리뷰_개수,평점,주소,영업시간,부가서비스,방문자_리뷰_글,개인리뷰
0,마르셀,여의도역,영등포구,"카페,디저트",132,287,4.64,서울 영등포구 여의대로 8 여의도파크센터 상가 지하1층,매일 18:00 - 02:00,"포장, 예약, 포장, 무선 인터넷, 남/녀 화장실 구분","인테리어가 멋져요: 40, 대화하기 좋아요: 31, 친절해요: 24, 매장이 청결해...",분위기와 조명이 남다른 곳이에요 또오고 싶어요 추천합 니다 음악도 감미롭고 좋아서 ...
1,오복수산대구,여의도역,영등포구,한식,232,103,4.63,서울 영등포구 의사당대로 83 오투타워 지하1층,"월 11:00 - 21:30 15:00 - 17:00 브레이크타임, 화 11:00 ...","단체석, 포장, 예약, 무선 인터넷","음식이 맛있어요: 154, 재료가 신선해요: 94, 매장이 청결해요: 68, 친절해...",맛있었어요 특히 8인룸은 조용하고 쾌적합니다 다만 8시 넘어도 품절된 메뉴가 많아서...
2,진주집,여의도역,영등포구,국수,"8,568","5,623",4.4,서울 영등포구 국제금융로6길 33 지하 1층,"월 10:00 - 20:00, 화 10:00 - 20:00, 수 10:00 - 20...","단체석, 포장, 예약","음식이 맛있어요: 3756, 양이 많아요: 1076, 매장이 넓어요: 1009, 특...",여름이면 꼭 가는 집이에요. 항상 성인2 아이 한명 가서 종류별로 다 시켜먹는데요....
3,옥된장 여의도점,여의도역,영등포구,한식,379,461,,"서울 영등포구 의사당대로 108 1층 102호, 104호","일 11:00 - 22:00, 월 11:00 - 01:00 14:30 - 16:00...",포장,"음식이 맛있어요: 176, 친절해요: 134, 재료가 신선해요: 123, 인테리어가...",수육전골 먹었는데 시원한 전골에 미나리가 들어가서 마 지막에 올라오는 향긋한 맛이 ...
4,미도인 파이낸스여의도,여의도역,영등포구,양식,"2,106","2,061",4.5,"서울 영등포구 국제금융로2길 32 파이낸스타워 2층 202, 203호","일 11:00 - 21:00 20:00 라스트오더, 월 10:40 - 21:00 2...",포장,"음식이 맛있어요: 1043, 친절해요: 402, 매장이 청결해요: 346, 인테리어...",단호박스프 띠용 완전 신세계 눈이떠지는맛이예요 ㅋ.ㅋ 마리네이드 토마토샐러드 무조건...
...,...,...,...,...,...,...,...,...,...,...,...,...
289,장독대김치찌개,샛강역,영등포구,한식,642,102,4.14,서울 영등포구 국제금융로 78 홍우빌딩,매일 00:00 - 24:00,,"음식이 맛있어요: 237, 양이 많아요: 112, 가성비가 좋아요: 94, 친절해요...",24시간 하는 가게여서 일이 늦게끝날때 밥한끼 먹으러가 면 좋은 곳이지만 혼밥 하러...
290,파파호 여의도점,샛강역,영등포구,베트남음식,445,98,4.33,서울 영등포구 국제금융로 78 홍우빌딩 1층,"일 11:00 - 15:00 14:30 라스트오더, 월 11:00 - 21:00 1...","포장, 예약, 무선 인터넷, 남/녀 화장실 구분","음식이 맛있어요: 185, 친절해요: 50, 재료가 신선해요: 44, 양이 많아요:...",쌀국수랑 볶음밥이 거의 1.5인분이라 항상 맛있게 먹고 있습니다! 반미도 빵이 두툼...
291,티에이씨,여의도역,영등포구,케이크전문,89,45,4.73,서울 영등포구 국제금융로6길 17 지하1층 15호,"월 10:00 - 18:00, 화 10:00 - 18:00, 수 10:00 - 18...",,"디저트가 맛있어요: 14, 친절해요: 11, 선물하기 좋아요: 10, 특별한 메뉴가...",의도치 않게 요구사항이 이것저것 있었는데 친절하게 응 대해주셨어요! 케익도 부탁드린...
292,화목순대국,샛강역,영등포구,"순대,순댓국","2,221","1,676",4.31,서울 영등포구 여의대방로 383 1F (경도상가 1층),"월 10:30 - 22:00 14:30 - 17:30 브레이크타임, 화 10:30 ...","단체석, 예약","음식이 맛있어요: 1156, 양이 많아요: 428, 특별한 메뉴가 있어요: 388,...",여긴 항상 오픈 전부터 대기가 있습니다. 첫 입장에 실패 하면 기본 30분 기다리셔...


In [26]:
df.to_csv('../data/여의도역_상세리뷰포함.csv', encoding='utf-8')